# 파이썬 데코레이터

데코레이터를 이야기 하기에 앞서 클래스를 좀 살펴보도록 하겠습니다.

파이썬에서 어떤 클래스에서 클래스 인스턴스 함수를 만들때는 항상
첫번째 패러미터로 self가 옵니다.

``` python
myCls:
	myF1(self, p1,p2):
		print('in myF1(p1="%s",p2="%s")' % (p1,p2))
```

위와 같은 경우 myF1의 첫번째 패러미터로 항상 self가 옵니다.
C++, C# 등에서와 같이 this 함수와 같은 의미로 이해하면 되겠지요.

헌데 다른 OOP의 경우에서 myCls 클래스의 static 함수개념이 있을 수 있습니다.
이런 경우는 

```python
myCls:
	myF2(p1,p2):
		print('in myF2(p1="%s",p2="%s")' % (p1,p2))
```

위의 myF2 함수와 같이 첫번째 패러미터로 self 가 존재하지 않습니다만,
여전히 p1으로 self 개체가 넘어가는 오류가 발생합니다.
이런 경우는

```python
myCls:
	@staticmethod
	myF2(p1,p2):
		print('in myF2(p1="%s",p2="%s")' % (p1,p2))
```

@staticmethod 라는 것을 이용하는데요,
이런 식으로 함수 앞에다 무언가 @ 로 시작하는 무언가를 호출하는 것을
파이썬에서 데코레이터라고 합니다.

[PEP (Python Enhancement Proposal) 318](https://www.python.org/dev/peps/pep-0318/)에서 제안되어 있습니다.

다음에는 다른 주제 이지만 get_param 이라는 함수의 패러미터를 확인해 보겠습니다.

In [1]:
#==========================================================================
def get_param(*args, **kwargs):
    sl = []
    for arg in args:
        if isinstance(arg,str):
            sl.append('%s:"%s"' % (type(arg).__name__,arg))
        else:
            sl.append('%s:%s' % (type(arg).__name__, arg))
    for k,v in kwargs.items():
        if isinstance(v,str):
            sl.append('%s=%s:"%s"' % (str(k),type(v).__name__,v))
        else:
            sl.append('%s=%s:%s' % (str(k),type(v).__name__,v))
    return ','.join(sl)

파이썬에서 모든 패러미터는 unnamed parameter와 named parameter로 구분되는데,
모든 가변 변수는 위와 같이 (\*args, \*\*kwargs) 로 모든 변수를 받아 볼 수 있습니다.
위의 get_param은 그런 모든 패러미터를 출력하는 것입니다.

다음에는 또 다른 함수를 만들어 봅니다.

In [2]:
###############################################################################
def entryExit(original_function):
    def new_function(*args, **kwargs):
        print(">>>Entering", original_function.__name__, '(%s)' % get_param(*args, **kwargs))
        original_function(*args, **kwargs)
        print("<<<Exited", original_function.__name__)
    return new_function

패러미터로 받은 original_function은 함수 포인터와 유사한 개념이라고 할 때,
그 안에 선언된 new_function은 일반 패러미터를 받아 출력하고 원본 함수를 다시 호출하는 것입니다.

드디어 데코레이터를 사용해 보겠습니다.

In [3]:
@entryExit
def hello(targetName=None):
    if targetName:
        print("\tHello, " +  targetName +"!")
    else:
        print("\tHello, world!")

또한 위와 같이 데코레이터를 가진 hello를 호출하면,

In [4]:
hello("Earth")

>>>Entering hello (str:"Earth")
	Hello, Earth!
<<<Exited hello


라고 결과가 나타납니다.

어떤 패러미터를 넣던지 위와 유사한 결과를 나타내줍니다.
위와 같은 것을 함수 데코레이터라 말합니다.

다음은,

In [5]:
class EntryExit(object):
    def __init__(self, f):
        self.f = f
    def __call__(self, *args, **kwargs):
        print(">>>Entering", self.f.__name__, '(%s)' % get_param(*args, **kwargs))
        self.f(*args, **kwargs)
        print("<<<Exited", self.f.__name__)

라는 클래스를 만들었습니다.

생성자에 패러미터로 f 라는 함수 포인터가 있다고 가정하고,
무언가가 호출될 때, __call__ 이 호출된다는 개념입니다.

위와 같은 것을

In [6]:
@EntryExit
def func1(*args, **kwargs):
    print("\tinside func1(%s)" % get_param(*args, **kwargs))

라고 선언을 하고,

다음과 같이 실행하면,

In [7]:
func1(2,'sss',foo='baa')

>>>Entering func1 (int:2,str:"sss",foo=str:"baa")
	inside func1(int:2,str:"sss",foo=str:"baa")
<<<Exited func1


위와 같은 entryExit와 같은 결과를 보여줍니다.
이와 같은 데코레이터를 클래스 데코레이터라고 합니다.

하지만 이는 클래스 안에서 self를 포함하여 호출할 때는 오류가 발생할 수 있습니다.
다음과 같은 코드가 있습니다.

In [8]:
class myClass(object):
    def myDecorator(original_function):
        def new_function(self, *args, **kwargs):
            print(">>>Entering", original_function.__name__, '(%s)' % get_param(*args, **kwargs))
            original_function(self, *args, **kwargs)
            print("<<<Exited", original_function.__name__)
        return new_function
    @entryExit
    def myFuncA(self, a, b, c='default'):
        print('\tmyFuncA(a="%s",b="%s",c="%s")' % (a,b,c))
    @myDecorator
    def myFuncB(self, a, b, c='default'):
        print('\tmyFuncA(a="%s",b="%s",c="%s")' % (a,b,c))

그리고 다음처럼 클래스를 생성하고,

In [9]:
mc = myClass()

다음과 같이 호출하면,

In [10]:
mc.myFuncA(1,2)

>>>Entering myFuncA (myClass:<__main__.myClass object at 0x110e14630>,int:1,int:2)
	myFuncA(a="1",b="2",c="default")
<<<Exited myFuncA


위와 같이 self까지 넘어와서 잘 호출된 것을 확인할 수 있습니다.

하지만,

```python
	@EntryExit
	def myFuncA(self, a, b, c='default'):
		print('\tmyFuncA(a="%s",b="%s",c="%s")' % (a,b,c))
```

와 같이 @entryExit 대신 @EntryExit를 호출하면 self 를 넘길 수 없어 오류가 발생하는데,
그와 같은 경우에는 위와 같이 self를 포함해서 넘기는 `myDecorator`와 같이 
함수를 지정하면 잘 동작합니다.

In [11]:
mc.myFuncB(3,'four')

>>>Entering myFuncB (int:3,str:"four")
	myFuncA(a="3",b="four",c="default")
<<<Exited myFuncB


이런 데코레이터를 이용하면 공통적인 공통적인 진입함수 또는  진출함수 등을 호출하는데 유용하게 이용될 수 있습니다.